In [ ]:
# ===== Enter Player's Name =====
def run():
    input_text = ""
    message = SlideMessage("Enter Your Name", font, HEIGHT // 2 - 200)
    confirming = False

    input_box = pygame.Rect(WIDTH // 2 - 250, HEIGHT // 2, 500, 80)
    enter_button = pygame.Rect(WIDTH // 2 - 100, HEIGHT // 2 + 120, 200, 60)

    active = False
    smaller_font = pygame.font.SysFont(None, 48)
    tiny_font = pygame.font.SysFont(None, 36)

    while True:
        screen.blit(background_img, (0, 0))
        mouse_pos = pygame.mouse.get_pos()
        hover_return = return_rect.collidepoint(mouse_pos)
        screen.blit(return_icon, return_rect)

        # 提示文字最先畫
        message.update()
        message.draw(screen)

        # 輸入框上方說明文字
        player_index = 1
        label_surface = small_font.render(f"Player {player_index}'s Name", True, BLACK)
        screen.blit(label_surface, (input_box.x, input_box.y - 100))

        # 輸入框背景
        pygame.draw.rect(screen, WHITE, input_box, border_radius=10)

        # 輸入框外框
        input_hover = input_box.collidepoint(mouse_pos)
        input_color = LIGHT_BLUE if active or input_hover else BLACK
        pygame.draw.rect(screen, input_color, input_box, 3, border_radius=10)

        # 顯示輸入文字
        name_surface = small_font.render(input_text, True, BLACK)
        text_y = input_box.y + (input_box.height - name_surface.get_height()) // 2
        screen.blit(name_surface, (input_box.x + 10, text_y))

        # === 閃爍游標 ===
        if active:
            cursor_x = input_box.x + 10 + name_surface.get_width() + 2
            cursor_y = text_y
            cursor_height = name_surface.get_height()

            if (pygame.time.get_ticks() // 500) % 2 == 0:
                pygame.draw.line(screen, BLACK, (cursor_x, cursor_y), (cursor_x, cursor_y + cursor_height), 2)

        # Enter 按鈕
        pygame.draw.rect(screen, LIGHT_BLUE, enter_button, border_radius=10)
        pygame.draw.rect(screen, BLACK, enter_button, 3, border_radius=10)
        enter_text = small_font.render("Enter", True, BLACK)
        screen.blit(enter_text, enter_text.get_rect(center=enter_button.center))

        # 返回提示泡泡
        if hover_return:
            draw_tooltip("Back", (return_rect.left + 70, return_rect.bottom - 50))

        # 如果正在確認，就顯示灰色遮罩 + 確認框
        if confirming:
            confirm_lines = [
                f"You are {input_text}!",
                "Are you sure? You won’t be able to change this later!"
            ]
            line_surfaces = [smaller_font.render(line, True, BLACK) for line in confirm_lines]

            max_width = max(surface.get_width() for surface in line_surfaces)
            total_text_height = sum(surface.get_height() for surface in line_surfaces) + (len(line_surfaces) - 1) * 10

            confirm_box_width = max_width + 80
            confirm_box_height = total_text_height + 150
            confirm_box = pygame.Rect(
                WIDTH // 2 - confirm_box_width // 2,
                HEIGHT // 2 - confirm_box_height // 2,
                confirm_box_width,
                confirm_box_height
            )
            confirm_ok = pygame.Rect(WIDTH // 2 - 120, confirm_box.centery + 20, 100, 50)
            confirm_cancel = pygame.Rect(WIDTH // 2 + 20, confirm_box.centery + 20, 100, 50)

            overlay = pygame.Surface((WIDTH, HEIGHT), pygame.SRCALPHA)
            overlay.fill((0, 0, 0, 160))
            screen.blit(overlay, (0, 0))

            pygame.draw.rect(screen, WHITE, confirm_box, border_radius=20)
            pygame.draw.rect(screen, BLACK, confirm_box, 4, border_radius=20)

            start_y = confirm_box.top + 40
            for i, surface in enumerate(line_surfaces):
                line_x = (WIDTH - surface.get_width()) // 2
                line_y = start_y + i * (surface.get_height() + 10)
                screen.blit(surface, (line_x, line_y))

            pygame.draw.rect(screen, LIGHT_BLUE, confirm_ok, border_radius=8)
            pygame.draw.rect(screen, BLACK, confirm_ok, 2, border_radius=8)
            ok_text = tiny_font.render("Confirm", True, BLACK)
            screen.blit(ok_text, ok_text.get_rect(center=confirm_ok.center))

            pygame.draw.rect(screen, GRAY, confirm_cancel, border_radius=8)
            pygame.draw.rect(screen, BLACK, confirm_cancel, 2, border_radius=8)
            cancel_text = tiny_font.render("Cancel", True, BLACK)
            screen.blit(cancel_text, cancel_text.get_rect(center=confirm_cancel.center))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if hover_return:
                    click_sound.play()
                    return None

                if confirming:
                    if confirm_ok.collidepoint(event.pos):
                        click_sound.play()
                        print(f"Player name confirmed: {input_text}")
                        return input_text
                    elif confirm_cancel.collidepoint(event.pos):
                        click_sound.play()
                        confirming = False
                else:
                    if input_box.collidepoint(event.pos):
                        active = True
                    else:
                        active = False

                    if enter_button.collidepoint(event.pos):
                        click_sound.play()
                        confirming = True

            elif event.type == pygame.KEYDOWN:
                if not confirming:
                    if active:
                        if event.key == pygame.K_RETURN:
                            confirming = True
                            click_sound.play()
                        elif event.key == pygame.K_BACKSPACE:
                            input_text = input_text[:-1]
                        else:
                            if len(input_text) < 20:
                                input_text += event.unicode
                elif confirming: # 當確認視窗開啟時，按 Enter 鍵就等於點擊 Confirm
                    if event.key == pygame.K_RETURN:
                        click_sound.play()
                        print(f"Player name confirmed: {input_text}")
                        return input_text

        pygame.display.update()
        clock.tick(60)

if __name__ == "__main__":
    run()
